<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Stance_Veracity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import fastai

In [0]:
from fastai.text import * 

In [0]:
def read_stance(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
# For stance
train = read_stance('train_stance.csv', True)
test = read_stance('test_stance.csv')

In [0]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [7]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [8]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [11]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.622311,4.717131,0.226027,00:04


In [12]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.476518,4.045491,0.293214,00:04


In [13]:
learn.predict("This is a review about", n_words=10)

'This is a review about taking un @charlescwcooke rides . And she shows them'

In [0]:
learn.save_encoder('ft_enc')

In [15]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (3389 items)
x: TextList
xxbos @genlady9 @rt_com xxmaj right a xxunk happening in xxmaj france and you post this . xxmaj this is the reason the world hates xxmaj islam .,xxbos @philserrin i think you missed @larryelder point . xxmaj where 's the xxunk over b - on - b killings & & shooting that xxunk from xxmaj chicago on daily basis ?,xxbos @kingpin7666 @cnnvideo you missed the part where he xxunk in and takes these cigars ? xxmaj xxunk them to his friend and takes some him self . xxup xxunk,xxbos xxmaj in 2009 , # xxmaj ferguson police beat a man and charged him for bleeding on them : http : / / t.co / xxunk,xxbos @max843 @danteb4u @alasscan _ xxmaj police lied ! xxmaj chief now says xxmaj xxunk 's friend was xxup not involved in any robbery . http : / / t.co / xxunk
y: CategoryList
Support,Support,Comment,Comment,Support
Path: .;

Valid: LabelList (848 items)
x: TextList
xxbos @tom_sveb @spants @rui_xu @flavius217 xxmaj not releasi

In [16]:
data_clas.show_batch()

text,target
xxbos .@marrty96 xxup and xxup yet xxup its xxup not xxup on / xxup in xxup the xxup tweet / xxup headline . xxup as xxup usual . & & xxup dont xxup fucking xxup tell xxup me xxup to xxup xxunk . \n xxup this xxup is xxup the xxup same xxup network xxup that xxup reported xxup xxunk,Support
xxbos @michaelskolnik @ac_bowen # xxmaj cleveland a xxmaj cop xxmaj shot a xxunk xxmaj old xxmaj in xxmaj the \n xxmaj head ! # xxmaj black # xxmaj unarmed # xxunk xxmaj cop xxmaj said xxmaj he # xxup thought xxmaj he \n xxmaj saw a xxmaj gun ! ! !,Support
xxbos “ @independent : xxmaj up to 20 held hostage in xxmaj sydney xxmaj lindt xxmaj cafe siege http : / / t.co / xxunk http : / / t.co / xxunk ” xxmaj oh . xxmaj my . xxmaj god . i am xxup sick !,Comment
xxbos “ xxup @davidmdraiman : .@cnni xxup its a xxup kosher xxup jewish xxup supermarket . xxup they xxup chose xxup it xxup for a xxup reason . xxup you xxup conveniently xxup left xxup that xxup out # jesuisjuif ” < too true !,Support
"xxbos @theroot xxunk @thedailybeast xxup wow ! xxup fantastic xxup article ! xxup rt & & xxup rt ! xxup fbi xxup needs xxup to xxup xxunk xxup this xxup one xxup also ! xxup no xxup justice , xxup no xxup peace !",Comment


In [17]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.921521,0.858705,0.709906,00:14


In [18]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.925615,0.855451,0.702830,00:15


In [19]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.838637,0.832793,0.713443,00:17
1,0.843511,0.835950,0.719340,00:17
2,0.812515,0.840600,0.724057,00:16
3,0.742215,0.867513,0.709906,00:17
4,0.697888,0.890471,0.700472,00:17
5,0.599990,0.929269,0.709906,00:15
6,0.597894,0.968694,0.687500,00:17
7,0.459178,0.958573,0.691038,00:17
8,0.459018,0.961888,0.691038,00:17
9,0.417038,0.969293,0.695755,00:17


In [0]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [22]:
accuracy_score(test['label'], l)

0.7295373665480427

In [23]:
f1_score(test['label'], l, average='macro')

0.5710479796379834